In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import matplotlib
# To show diagrams inline.
%matplotlib inline

In [233]:
df_train = pd.read_excel('train_data.xlsx')
df_test = pd.read_excel('test_data.xlsx')

In [234]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [235]:
df_test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [236]:
df = pd.concat([df_train, df_test], axis=0, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.1+ MB


In [237]:
print(df_train.shape, df_test.shape, df.shape)

(10683, 11) (2671, 10) (13354, 11)


In [238]:
# Create derived features for Date_Of_Journey
date = df['Date_of_Journey'].str.split('/', expand=True)
date.columns = ['Day', 'Month', 'Year']
df['Day'] = date['Day'].astype(int)
df['Month'] = date['Month'].astype(int)
df['Year'] = date['Year'].astype(int)
df.drop(['Date_of_Journey'], axis=1, inplace=True)


In [239]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Day,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


In [240]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Dep_Time         13354 non-null  object 
 5   Arrival_Time     13354 non-null  object 
 6   Duration         13354 non-null  object 
 7   Total_Stops      13353 non-null  object 
 8   Additional_Info  13354 non-null  object 
 9   Price            10683 non-null  float64
 10  Day              13354 non-null  int64  
 11  Month            13354 non-null  int64  
 12  Year             13354 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 1.3+ MB


In [241]:
df.isnull().sum()

Airline               0
Source                0
Destination           0
Route                 1
Dep_Time              0
Arrival_Time          0
Duration              0
Total_Stops           1
Additional_Info       0
Price              2671
Day                   0
Month                 0
Year                  0
dtype: int64

# Observation
1. Day, Month and Year are now integers.
2. Route and Total_Stops have one null value.
3. Null value of Price is because of test data

In [242]:
# Focus on Arrival_Time and Departure_Time
df['Arrival_Time'] = df['Arrival_Time'].str.split(' ').str[0]
# Above can also be done using lambda function
# df['Arrival_Time'] = df['Arrival_Time'].apply(lambda x: x.split(' ')[0])


In [243]:
# Convert Arrival_Time to hours and minutes
df['Arrival_Hour'] = df['Arrival_Time'].str.split(':').str[0].astype(int)
df['Arrival_Minute'] = df['Arrival_Time'].str.split(':').str[1].astype(int)
# Drop Arrival_Time column
df.drop(['Arrival_Time'], axis=1, inplace=True)
df.head()

,Airline,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Day,Month,Year,Arrival_Hour,Arrival_Minute
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,19h,2 stops,No info,13882.0,9,6,2019,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35


In [244]:
# Convert Dep_Time to hours and minutes
df['Departure_Hour'] = df['Dep_Time'].str.split(':').str[0].astype(int)
df['Departure_Minute'] = df['Dep_Time'].str.split(':').str[1].astype(int)
# Drop Dep_Time column
df.drop(['Dep_Time'], axis=1, inplace=True)
df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Day,Month,Year,Arrival_Hour,Arrival_Minute,Departure_Hour,Departure_Minute
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882.0,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35,16,50


### Observation
1. We can drop Routes as we already have Source, Destination and Total_Stops. To predict the price, Source, Destination and Total_Stops would be required.
2. We would need to convert Total_Stops into integers.

In [245]:
# Drop Route column
df.drop(['Route'], axis=1, inplace=True)

In [246]:
# Working on Total_Stops
# Checking for null values in Total_Stops column
df['Total_Stops'].isnull().sum()
# Finding out the null values in Total_Stops column
df[df['Total_Stops'].isnull()]

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Day,Month,Year,Arrival_Hour,Arrival_Minute,Departure_Hour,Departure_Minute
9039,Air India,Delhi,Cochin,23h 40m,NaN,No info,7480.0,6,5,2019,9,25,9,45


In [247]:
# Filling null values with 'non-stop'. Since it is just one row, so this won't make much difference
df['Total_Stops'].fillna('non-stop', inplace=True)
df['Total_Stops'] = df['Total_Stops'].map({'non-stop': 0, '1 stop': 1, '2 stops': 2, '3 stops': 3, '4 stops': 4})
df.head()

C:\Users\akashanup\AppData\Local\Temp\ipykernel_30184\2987119114.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Total_Stops'].fillna('non-stop', inplace=True)


,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Day,Month,Year,Arrival_Hour,Arrival_Minute,Departure_Hour,Departure_Minute
0,IndiGo,Banglore,New Delhi,2h 50m,0,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,19h,2,No info,13882.0,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,5h 25m,1,No info,6218.0,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,4h 45m,1,No info,13302.0,1,3,2019,21,35,16,50


In [248]:
df['Duration'].unique()

array(['2h 50m', '7h 25m', '19h', '5h 25m', '4h 45m', '2h 25m', '15h 30m',
       '21h 5m', '25h 30m', '7h 50m', '13h 15m', '2h 35m', '2h 15m',
       '12h 10m', '26h 35m', '4h 30m', '22h 35m', '23h', '20h 35m',
       '5h 10m', '15h 20m', '2h 55m', '13h 20m', '15h 10m', '5h 45m',
       '5h 55m', '13h 25m', '22h', '5h 30m', '10h 25m', '5h 15m',
       '2h 30m', '6h 15m', '11h 55m', '11h 5m', '8h 30m', '22h 5m',
       '2h 45m', '12h', '16h 5m', '19h 55m', '3h 15m', '25h 20m', '3h',
       '16h 15m', '15h 5m', '6h 30m', '25h 5m', '12h 25m', '27h 20m',
       '10h 15m', '10h 30m', '1h 30m', '1h 25m', '26h 30m', '7h 20m',
       '13h 30m', '5h', '19h 5m', '14h 50m', '2h 40m', '22h 10m',
       '9h 35m', '10h', '21h 20m', '18h 45m', '12h 20m', '18h', '9h 15m',
       '17h 30m', '16h 35m', '12h 15m', '7h 30m', '24h', '8h 55m',
       '7h 10m', '14h 30m', '30h 20m', '15h', '12h 45m', '10h 10m',
       '15h 25m', '14h 5m', '20h 15m', '23h 10m', '18h 10m', '16h',
       '2h 20m', '8h', '16h 5

### Observation
1. There are three types of values for Duration-
  - **h **m
  - **h
  - **m
2. We need to handle this carefully.

In [249]:
duration = df['Duration'].str.split(' ')
duration_hours_and_minutes = pd.DataFrame([dur for dur in duration if len(dur) == 2])
duration_hours_or_minutes = pd.DataFrame([dur for dur in duration if len(dur) == 1])

# Verify the filtered data
duration_hours_and_minutes.head(), duration_hours_or_minutes.head()

(    0    1
 0  2h  50m
 1  7h  25m
 2  5h  25m
 3  4h  45m
 4  2h  25m,
      0
 0  19h
 1  23h
 2  22h
 3  12h
 4   3h)

In [250]:
duration_hours_or_minutes[0].unique()

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h', '5m'],
      dtype=object)

### Observation
1. There are instances where flight duration is only 5 minutes.
2. Practically these values aren't possible so we should drop them.

In [251]:
duration_hours_or_minutes[duration_hours_or_minutes[0] == '5m']

,0
595,5m
1285,5m


In [252]:
# Drop the rows with only 5 minutes of duration
duration_hours_or_minutes.drop(595, inplace=True, axis=0)
duration_hours_or_minutes.drop(1285, inplace=True, axis=0)
df.drop([595, 1285], inplace=True, axis=0)
duration_hours_or_minutes[0].unique()

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [253]:
# Appending 0m to rows with only hours in duration_hours_or_minutes
duration_hours_or_minutes[1] = ' 0m'
duration_hours_or_minutes.head()

,0,1
0,19h,0m
1,23h,0m
2,22h,0m
3,12h,0m
4,3h,0m


In [254]:
# Appending duration_hours_or_minutes to duration_hours_and_minutes
duration_hours_and_minutes = pd.concat([duration_hours_and_minutes, duration_hours_or_minutes], axis=0, ignore_index=True)
print(df['Duration'].shape, duration_hours_and_minutes.shape)

(13352,) (13352, 2)


In [255]:
# Convert duration_hours_and_minutes to hours and minutes
duration_hour = duration_hours_and_minutes[0]
duration_minute = duration_hours_and_minutes[1]
duration_hour.head(), duration_minute.head()

(0    2h
 1    7h
 2    5h
 3    4h
 4    2h
 Name: 0, dtype: object,
 0    50m
 1    25m
 2    25m
 3    45m
 4    25m
 Name: 1, dtype: object)

In [256]:
# Merging duration_hours and duration_minutes
df['Duration_Hour'] = duration_hour.str.split('h').str[0].astype(int)
df['Duration_Minute'] = duration_minute.str.split('m').str[0].astype(int)
# Drop Duration column
df.drop(['Duration'], axis=1, inplace=True)
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Day,Month,Year,Arrival_Hour,Arrival_Minute,Departure_Hour,Departure_Minute,Duration_Hour,Duration_Minute
0,IndiGo,Banglore,New Delhi,0,No info,3897.0,24,3,2019,1,10,22,20,2.0,50.0
1,Air India,Kolkata,Banglore,2,No info,7662.0,1,5,2019,13,15,5,50,7.0,25.0
2,Jet Airways,Delhi,Cochin,2,No info,13882.0,9,6,2019,4,25,9,25,5.0,25.0
3,IndiGo,Kolkata,Banglore,1,No info,6218.0,12,5,2019,23,30,18,5,4.0,45.0
4,IndiGo,Banglore,New Delhi,1,No info,13302.0,1,3,2019,21,35,16,50,2.0,25.0


In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13352 entries, 0 to 13353
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Airline           13352 non-null  object 
 1   Source            13352 non-null  object 
 2   Destination       13352 non-null  object 
 3   Total_Stops       13352 non-null  int64  
 4   Additional_Info   13352 non-null  object 
 5   Price             10681 non-null  float64
 6   Day               13352 non-null  int64  
 7   Month             13352 non-null  int64  
 8   Year              13352 non-null  int64  
 9   Arrival_Hour      13352 non-null  int64  
 10  Arrival_Minute    13352 non-null  int64  
 11  Departure_Hour    13352 non-null  int64  
 12  Departure_Minute  13352 non-null  int64  
 13  Duration_Hour     13350 non-null  float64
 14  Duration_Minute   13350 non-null  float64
dtypes: float64(3), int64(8), object(4)
memory usage: 1.6+ MB


In [258]:
# Convert categorical variables to numerical variables using label encoding
print(df['Airline'].unique(), df['Source'].unique(), df['Destination'].unique(), df['Additional_Info'].unique())

['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet'] ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai'] ['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad'] ['No info' 'In-flight meal not included' 'No check-in baggage included'
 '1 Short layover' 'No Info' '1 Long layover' 'Change airports'
 'Business class' 'Red-eye flight' '2 Long layover']


In [259]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [260]:
df['Airline'] = le.fit_transform(df['Airline'])
df['Source'] = le.fit_transform(df['Source'])
df['Destination'] = le.fit_transform(df['Destination'])
df['Additional_Info'] = le.fit_transform(df['Additional_Info'])
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Day,Month,Year,Arrival_Hour,Arrival_Minute,Departure_Hour,Departure_Minute,Duration_Hour,Duration_Minute
0,3,0,5,0,8,3897.0,24,3,2019,1,10,22,20,2.0,50.0
1,1,3,0,2,8,7662.0,1,5,2019,13,15,5,50,7.0,25.0
2,4,2,1,2,8,13882.0,9,6,2019,4,25,9,25,5.0,25.0
3,3,3,0,1,8,6218.0,12,5,2019,23,30,18,5,4.0,45.0
4,3,0,5,1,8,13302.0,1,3,2019,21,35,16,50,2.0,25.0


In [ ]:
# We can also use OneHotEncoder for categorical variables
# from sklearn.preprocessing import OneHotEncoder

# ohe = OneHotEncoder(sparse=False, drop='first')
# df_airline = ohe.fit_transform(df[['Airline']])
# df_source = ohe.fit_transform(df[['Source']])
# df_destination = ohe.fit_transform(df[['Destination']])
# df_additional_info = ohe.fit_transform(df[['Additional_Info']])
# df_airline = pd.DataFrame(df_airline, columns=ohe.get_feature_names_out(['Airline']))
# df_source = pd.DataFrame(df_source, columns=ohe.get_feature_names_out(['Source']))
# df_destination = pd.DataFrame(df_destination, columns=ohe.get_feature_names_out(['Destination']))
# df_additional_info = pd.DataFrame(df_additional_info, columns=ohe.get_feature_names_out(['Additional_Info']))